In [1]:
import numpy as np
import pandas as pd


def Mass(R):
    """Empirical mass-radius relation from Eker+2018.
    From Table 5.
    
    Parameters:
    -----------
    R : float
        radius in solar radii
    
    Return:
    -------
    stellar mass in solar masses
    """
    c = .075
    b = .479
    a = .438
    return (-b + np.sqrt(b**2 - 4. * .438 * (.075 - R))) / (2. * .438)

In [122]:
# Load infor on effective temperature and radius

r147 = pd.read_csv("../StellarParameters/luminosities/ngc6774_luminosities.csv")
rm67 = pd.read_csv("../StellarParameters/luminosities/ngc2682_luminosities.csv")

row147 = r147.loc[r147.EPIC.isin([219591752, 219601739, 219610232]),
                  ["EPIC","Teff_median","Teff_std", "Rstar", "e_Rstar"]]
rowm67 = rm67.loc[rm67.EPIC ==211434440,
                  ["EPIC","Teff_median","Teff_std", "Rstar", "e_Rstar"]]

TR = pd.concat([rowm67,row147])
TR

,EPIC,Teff_median,Teff_std,Rstar,e_Rstar
833,211434440.0,5188.269447,244.211866,0.803806,0.072904
59,219601739.0,5458.330000,175.000000,0.862231,0.062612
78,219610232.0,5226.132958,396.762087,0.813195,0.069936
80,219591752.0,3314.972737,183.320727,0.321764,0.107488


In [123]:
# calculate masses using empirical relations from Eker+2018
TR.EPIC = TR.EPIC.astype(int)
TR["M_mid"] = Mass(TR.Rstar)
TR["M_max"] = Mass(TR.Rstar + TR.e_Rstar)
TR["M_min"] = Mass(TR.Rstar - TR.e_Rstar)
TR["M_hide_max"] = TR.apply(lambda x: min(x.M_max - x.M_min, x.M_max / 2.),axis=1)

In [124]:
# Look up SpT from Mamajek+Pecaut 2013
SpT = {"EPIC" : [211434440, 219601739, 219610232, 219591752],
       "SpT" : ["K1","G8","K0.5","M3"], #from Teff Pecaut and Mamajek (2013)
       "SpT_hiding_max": ["M5.5","M6","M5.5","M3.5"], #from M_hide_max Pecaut and Mamajek (2013)
       "SpT_visible_min":["K2","K1","K2","M3.5"]} #from M_min, or equal to M_hide_max if equal-mass is possible

In [125]:
TR = TR.merge(pd.DataFrame(SpT), on="EPIC")
TR["binary"] = TR.apply(lambda x: f"{x.SpT_visible_min} + {x.SpT_hiding_max}",axis=1) 
TR

,EPIC,Teff_median,Teff_std,Rstar,e_Rstar,M_mid,M_max,M_min,M_hide_max,SpT,SpT_hiding_max,SpT_visible_min,binary
0,211434440,5188.269447,244.211866,0.803806,0.072904,0.854244,0.912437,0.793528,0.118909,K1,M5.5,K2,K2 + M5.5
1,219601739,5458.330000,175.000000,0.862231,0.062612,0.901065,0.949617,0.850828,0.098789,G8,M6,K1,K1 + M6
2,219610232,5226.132958,396.762087,0.813195,0.069936,0.861873,0.917451,0.804011,0.113441,K0.5,M5.5,K2,K2 + M5.5
3,219591752,3314.972737,183.320727,0.321764,0.107488,0.381842,0.505712,0.238675,0.252856,M3,M3.5,M3.5,M3.5 + M3.5


In [127]:
tr = TR[["EPIC","SpT","binary"]]
tr = tr.rename(index=str, columns={"SpT":"median SpT"})
print(tr.to_latex(escape=False,index=False))

\begin{tabular}{rll}
\toprule
      EPIC & median SpT &       binary \\
\midrule
 211434440 &         K1 &    K2 + M5.5 \\
 219601739 &         G8 &      K1 + M6 \\
 219610232 &       K0.5 &    K2 + M5.5 \\
 219591752 &         M3 &  M3.5 + M3.5 \\
\bottomrule
\end{tabular}



## Side notes

Masses from Huber+2016: 

ID| EPIC| M| e_M_upper| e_M_lower|
---|---|---|---|---|
R3| 219610232| 0.800| 	0.079| 	0.079|
R1| 219601739| 1.050| 	0.090| 	0.144|
M1| 211434440| 0.870| 	0.063| 	0.075| 	
